In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import html
import ast
import time
import pprint
import pandas as pd
import datetime
import csv
from bs4 import BeautifulSoup
url = "https://www.canberrahealthservices.act.gov.au/_designs/generic-modules/entitywaitingtimes/asset-listings/dynamic-wait-times-data-raw-asset-listing/_nocache"

In [5]:
# Set up Google Sheets API credentials
key = '../google_api_cred_key.json'
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(key, scope)
client = gspread.authorize(creds)

# Open the Google Sheet (use the name or URL of your sheet)
spreadsheet = client.open('wic_wait_times')
worksheet = spreadsheet.get_worksheet(0)  # Open the first sheet

In [ ]:
header_written = False

try:
    while True:
        try:
            # Attempt to request the URL with a timeout
            response = requests.get(url, timeout=10)  # Set timeout to 10 seconds
            response.raise_for_status()  # Raise an HTTPError for bad responses (4XX/5XX)
            
            # Check if response is not empty
            if not response.text.strip():
                print(f"{datetime.datetime.now()}: Empty response, retrying in 15 seconds...")
                time.sleep(15)
                continue

            # Parse the response
            soup = BeautifulSoup(response.text, 'html.parser')
            data_element = soup.find('div', id='divEntitiesJSON').get_text(strip=True)

            # If data_element is empty, retry after 15 seconds
            if not data_element:
                print(f"{datetime.datetime.now()}: No data found in response, retrying in 15 seconds...")
                time.sleep(15)
                continue

            # Convert the data to a dictionary
            to_dict = ast.literal_eval(data_element)

            # Process the data
            rows = []
            for entry in to_dict:
                id_ = entry['id']
                last_updated = entry['lastUpdated']
                clinic_type = entry['type']
                queue = next((x['queue'] for x in entry['waitTimes'] if x['name'] == 'Wait'), None)
                wait_time = next((x['duration'] for x in entry['waitTimes'] if x['name'] == 'Wait'), None)
                treat_time = next((x['duration'] for x in entry['waitTimes'] if x['name'] == 'Treat'), None)
                total_time = next((x['duration'] for x in entry['waitTimes'] if x['name'] == 'Total'), None)
                to_append = {
                    'id': id_,
                    'lastUpdated': last_updated,
                    'clinic_type': clinic_type,
                    'queue': queue,
                    'wait_time': wait_time,
                    'treat_time': treat_time,
                    'total_time': total_time
                }
                rows.append(to_append)

            # Convert rows to DataFrame
            df = pd.DataFrame(rows)

            # Append DataFrame to Google Sheet
            try:
                if not header_written:
                    worksheet.append_row(df.columns.tolist())  # Write headers to Google Sheet
                    header_written = True
    
                for row in df.itertuples(index=False, name=None):
                    worksheet.append_row(list(row))  # Append each row to Google Sheet

                print(f"{datetime.datetime.now()}: Successfully printed stats from ACT Health! Zzzz...")
                time.sleep(59)
            
            except Exception as google_sheets_exception:
                print(f"{datetime.datetime.now()}: Google sheets API error, sleeping")
                time.sleep(10)

        except requests.exceptions.Timeout:
            print(f"{datetime.datetime.now()}: Request timed out, retrying in 15 seconds...")
            time.sleep(15)

        except requests.exceptions.RequestException as e:
            print(f"{datetime.datetime.now()}: An error occurred: {e}")
            time.sleep(59)

except Exception as e:
    print(f"{datetime.datetime.now()}: Fatal error: {e}")
    raise e

2024-10-22 21:38:28.335592: Successfully printed stats from ACT Health! Zzzz...
2024-10-22 21:39:30.232363: Successfully printed stats from ACT Health! Zzzz...
2024-10-22 21:40:31.867530: Successfully printed stats from ACT Health! Zzzz...
2024-10-22 21:41:33.530474: Successfully printed stats from ACT Health! Zzzz...
2024-10-22 21:42:35.392292: Successfully printed stats from ACT Health! Zzzz...
2024-10-22 21:43:37.387000: Successfully printed stats from ACT Health! Zzzz...
2024-10-22 21:44:39.169818: Successfully printed stats from ACT Health! Zzzz...
2024-10-22 21:45:40.876790: Successfully printed stats from ACT Health! Zzzz...
